In [1]:
import os 
import pandas as pd

In [6]:
dataset_path = "audio/datasets/audio_speech_actors_01-24"
data = []


#Example format: "03-01-05-01-02-01-12.wav"
#Breakdown: modality - vocal_channel - emotion - intensity - statement - repetition - actor

In [8]:
for actor_folder in os.listdir(dataset_path):
    actor_path = os.path.join(dataset_path,actor_folder)
    
    if os.path.isdir(actor_path):
        for file_name in os.listdir(actor_path):
            if file_name.endswith(".wav"):
                parts = file_name.split("-")
                emotion = int(parts[2])
                actor_id = int(parts[-1].split(".")[0])
                file_path = os.path.join(actor_path,file_name)
                data.append({
                    "file_path" : file_path,
                    "emotion": emotion,
                    "actor": actor_id
                })
df = pd.DataFrame(data)
df.to_csv("ravdess_labelled_data.csv",index = False)
df.head()

,file_path,emotion,actor
0,audio/datasets/audio_speech_actors_01-24\Actor...,1,1
1,audio/datasets/audio_speech_actors_01-24\Actor...,1,1
2,audio/datasets/audio_speech_actors_01-24\Actor...,1,1
3,audio/datasets/audio_speech_actors_01-24\Actor...,1,1
4,audio/datasets/audio_speech_actors_01-24\Actor...,2,1


In [9]:
#to encode emotionnumber to emotion
# Mapping dictionary
# emotion_map = {
#     1: "neutral",
#     2: "calm",
#     3: "happy",
#     4: "sad",
#     5: "angry",
#     6: "fearful",
#     7: "disgust",
#     8: "surprised"
# }

# Apply the mapping to the 'emotion' column
# df['emotion'] = df['emotion'].map(emotion_map)
# df.to_csv("ravdess_data.csv", index=False)


In [7]:
import librosa
import numpy as np

def extract_features(file_path):
    y, sr = librosa.load(file_path, duration=3, offset=0.5)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    return np.mean(mfcc.T, axis=0)

In [2]:
import pandas as pd
import os
df = pd.read_csv("ravdess_labelled_data.csv")
df.head()


,file_path,emotion,actor
0,audio/datasets/audio_speech_actors_01-24\Actor...,1,1
1,audio/datasets/audio_speech_actors_01-24\Actor...,1,1
2,audio/datasets/audio_speech_actors_01-24\Actor...,1,1
3,audio/datasets/audio_speech_actors_01-24\Actor...,1,1
4,audio/datasets/audio_speech_actors_01-24\Actor...,2,1


In [4]:
print(df.columns)

Index(['file_path', 'emotion', 'actor'], dtype='object')


In [8]:
features = []
labels = []

for index, row in df.iterrows():
    mfccs = extract_features(row['file_path'])
    features.append(mfccs)
    labels.append(row['emotion'])

X = np.array(features)
y = np.array(labels)


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = RandomForestClassifier()
model.fit(X_train, y_train)

preds = model.predict(X_test)
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           1       0.73      0.40      0.52        20
           2       0.37      0.81      0.51        27
           3       0.44      0.27      0.34        44
           4       0.60      0.45      0.51        40
           5       0.67      0.75      0.71        32
           6       0.71      0.66      0.68        38
           7       0.67      0.63      0.65        46
           8       0.53      0.61      0.57        41

    accuracy                           0.57       288
   macro avg       0.59      0.57      0.56       288
weighted avg       0.59      0.57      0.56       288

